# Avaliando o enfrentamento político à crise climática nas eleições municipais do Brasil em 2024

Instalação das dependências:

In [ ]:
!pip install -r requirements.txt

O código abaixo gera os arquivos:
- `analise-llm.log` com a resposta do modelo para cada página de cada plano de governo; e 
- `resultados.json` com a informação de quais páginas continham referências a propostas de prevenção de desastres climáticos em cada documento.

**Importante:** a execução pode ser bastante demorada de acordo com a configuração da máquina que executa o LLM.

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from ollama import ResponseError
import pymupdf
import os
import json
from tqdm.notebook import tqdm
import logging

h = logging.FileHandler(filename='analise-llm.log')
h.setLevel(logging.INFO)
logger = logging.getLogger(__name__)
logger.addHandler(h)
logger.setLevel(logging.INFO)

logger.info('Iniciando execução.')

model = OllamaLLM(model="llama3.1:70b")

resultados = {}

top_dir = './pdfs'

prompt_template = """O texto a seguir, delimitado por triplos parênteses, foi extraído do plano de governo de um candidato a prefeito.
O texto contém alguma referência a uma proposta de plano de prevenção ou mitigação de riscos relacionados a eventos climáticos extremos (e.g.: inundações, deslizamentos, secas)?
Caso o texto não mencione o assunto, responda apenas NÃO
Caso o texto aborde o assunto, responda: SIM. trecho:<citação>
onde <citação> deve ser substituído por um trecho da frase que referencia o assunto.
Ignore eventuais erros na formatação do texto e responda apenas como pedido.
(((
    {texto}
)))
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

chain = prompt | model

lista_cidades = os.listdir(top_dir)
try:
    for cidade in lista_cidades:
        res_cidade = {}
        print(f"Avaliando cidade de {cidade}")
        for pdf in os.listdir(os.path.join(top_dir, cidade)):
            refs_clima = []
            print(f'Avaliando "{pdf}":')
            for page in tqdm(pymupdf.open(os.path.join(top_dir, cidade, pdf))):
                text = page.get_text()
                retries = 0
                while True:
                    try:
                        response = chain.invoke({'texto': text})
                        break
                    except ResponseError as e:
                        retries += 1
                        if retries > 2:
                            raise e
                logger.info(f"{cidade}/{pdf}/{page.number + 1}:{response}")
                if "SIM" in response:
                    refs_clima.append(page.number + 1)
            res_cidade[pdf] = refs_clima
        resultados[cidade] = res_cidade
finally:
    h.close()
    json.dump(resultados, fp=open("resultados.json", "w"), ensure_ascii=False)
    json.dumps(resultados, ensure_ascii=False)

## Exploração dos resultados

In [ ]:
!pip install pandas matplotlib

In [ ]:
import json
import pandas as pd

with open("out/resultados.json", 'r') as res_file:
    res_json = json.load(res_file)

readable_results = []
for cid, v in res_json.items():
    for doc, refs in v.items():
        readable_results.append({'município': cid, 'documento': doc, 'aborda_tema': bool(refs), 'páginas': refs})

resultados = pd.DataFrame(readable_results)

resultados

In [ ]:
unica_mencao = [['município', 'candidata(o)', 'proposta validada', 'justificativa'],
                ['Caracaraí', 'Gildeone Roque', 'N', 'meios de financiamento de sistema de detecção de tempestades, sem especificar finalidade'],
                ['Rio de Janeiro', 'Eduardo Paes', 'S', 'houve atenção mas condensada em uma página'],
                ['Batayporã', 'Germino Roz', 'N', 'apenas drenagem do pavimento'],
                ['São Luiz Paraitinga', 'Flavio Bonafe', 'N', 'drenagem para evitar acúmulo de água'],
                ['Itati', 'Jussara Bobsin', 'S', 'houve atenção mas condensada em uma página'],
                ['São Paulo', 'Pablo Marçal', 'N', 'revitalização de parques, sem menção ao enfrentamento de desastres'],
                ['Três Forquilhas', 'Bruno', 'S', 'enfrentamento sem reconhecimento do papel da crise climática'],
                ['Igarassu', 'Miguel Ricardo', 'S', 'enfrentamento sem reconhecimento do papel da crise climática'],
                ['Igarassu', 'Professora Elcione', 'S', 'houve atenção mas condensada em uma página'],
                ['Barreiros', 'Claudio da Cooates', 'S', 'enfrentamento sem reconhecimento do papel da crise climática'],
                ['Barreiros', 'Carlinhos da Pedreira', 'S', 'enfrentamento sem reconhecimento do papel da crise climática'],
                ['Jurema', 'Branco de Geraldo', 'S', 'enfrentamento sem reconhecimento do papel da crise climática'],
                ['Jurema', 'Léo Ramos', 'S', 'enfrentamento sem reconhecimento do papel da crise climática'],
                ['Ribeirão das Neves', 'Vicente Mendonca', 'S', 'enfrentamento sem reconhecimento do papel da crise climática'],
               ]

unica_mencao = pd.DataFrame(unica_mencao[1:], columns=unica_mencao[0])

unica_mencao
                

In [85]:
import os

dados_tse = pd.read_csv(os.path.join('dados_manuais', 'prefeituras_clima_dados_tse.csv'), sep=';')
resultados2 = resultados.merge(dados_tse, how='left', on=['município', 'documento'])
resultados = resultados2.merge(unica_mencao, how='left', on=['município', 'candidata(o)'])

resultados.loc[resultados['proposta validada'] == 'N', 'aborda_tema'] = False

resultados.drop(['documento', 'justificativa', 'proposta validada', 'páginas'], axis=1, inplace=True)

resultados

,município,aborda_tema,candidata(o),partido,coligação,resultado
0,Caracaraí,False,Diego Farias,PSDB,PDT / Federação PSDB CIDADANIA(PSDB/CIDADANIA),não eleita
1,Caracaraí,False,Diane Coelho,PP,PP / REPUBLICANOS / PODE,eleita
2,Caracaraí,False,Antônio Reis,MDB,MDB / PL,não eleita
3,Caracaraí,False,Gildeone Roque,NOVO,NOVO,não eleita
4,Rio de Janeiro,True,Alexandre Ramagem,PL,PL / MDB / REPUBLICANOS,não eleita
5,Rio de Janeiro,True,Eduardo Paes,PSD,PODE / PRD / DC / AGIR / SOLIDARIEDADE / AVANT...,eleita
6,Batayporã,False,Germino Roz,PSDB,PP / UNIÃO / PODE / Federação BRASIL DA ESPERA...,eleita
7,Batayporã,False,Edmilson Gaiseiro,MDB,PDT / REPUBLICANOS / SOLIDARIEDADE / MDB,não eleita
8,São Luiz Paraitinga,False,Flavio Bonafe,MDB,MDB / PSB / SOLIDARIEDADE / Federação BRASIL D...,não eleita
9,São Luiz Paraitinga,True,Antonio Ozorio,PSD,PSD / UNIÃO / PRD / PODE / Federação PSDB CIDA...,não eleita


### Exportando para Excel

In [ ]:
!pip install xlsxwriter

In [86]:
resultados.to_excel('prefeituras_clima_resumo_v2.xlsx', index=False)